In [1]:
import boto3

***Create S3 Bucket***

In [3]:
# Create an S3 client
s3 = boto3.client('s3')

In [ ]:
s3.list_buckets()

In [5]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created")

create_bucket('my-very-important-bucket-7542')

Bucket 'my-very-important-bucket-7542' created


***Upload data to the bucket***

In [8]:
import os

In [9]:
bucket_name = 'my-very-important-bucket-7542'

def upload_file(file_path, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)
    s3.upload_file(file_path, bucket_name, object_name)
    print(f"File '{file_path}' uploaded as '{object_name}'")

In [11]:
upload_file('data/DigiSoft_Privacy_Policy.pdf')

File 'data/DigiSoft_Privacy_Policy.pdf' uploaded as 'DigiSoft_Privacy_Policy.pdf'


In [12]:
def list_files(bucket_name):
    response = s3.list_objects_v2(Bucket=bucket_name)

    for obj in response.get('Contents', []):
        print(f" - {obj['Key']} (size: {obj['Size']} bytes)")

list_files(bucket_name)

 - DigiSoft_Privacy_Policy.pdf (size: 2289 bytes)


***Download S3 File to Local System***

In [13]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))

    s3.download_file(bucket_name, object_name, file_path)
    print(f"File '{object_name}' downloaded to '{file_path}'")

download_file('DigiSoft_Privacy_Policy.pdf', 'downloads/DigiSoft_Privacy_Policy.pdf')

File 'DigiSoft_Privacy_Policy.pdf' downloaded to 'downloads/DigiSoft_Privacy_Policy.pdf'


***Upload a folder to S3***

In [20]:
def upload_directory(directory_path, s3_prefix):
    for root, _, files in os.walk('data'):
        for file in files:
            file_path = os.path.join(root, file).replace('\\', '/')
            relpath = os.path.relpath(file_path, directory_path)
            s3_key = os.path.join(s3_prefix, relpath).replace('\\', '/')
            s3.upload_file(file_path, bucket_name, s3_key)
            print(f"Uploaded '{file_path}' to '{s3_key}'")

upload_directory('data', 'info_docs')

Uploaded 'data/DigiSoft_Privacy_Policy.pdf' to 'info_docs/DigiSoft_Privacy_Policy.pdf'
Uploaded 'data/DigiSoft_Product_and_Services.pdf' to 'info_docs/DigiSoft_Product_and_Services.pdf'
Uploaded 'data/DigiSoft_Security_and_Compliance.pdf' to 'info_docs/DigiSoft_Security_and_Compliance.pdf'
Uploaded 'data/DigiSoft_Support_and_Refund_Policy.pdf' to 'info_docs/DigiSoft_Support_and_Refund_Policy.pdf'
Uploaded 'data/DigiSoft_Terms_of_Service.pdf' to 'info_docs/DigiSoft_Terms_of_Service.pdf'
Uploaded 'data/all_data/general info/DigiSoft_Company_Overview.pdf' to 'info_docs/all_data/general info/DigiSoft_Company_Overview.pdf'
Uploaded 'data/all_data/terms and policies/DigiSoft_Privacy_Policy.pdf' to 'info_docs/all_data/terms and policies/DigiSoft_Privacy_Policy.pdf'
Uploaded 'data/all_data/terms and policies/DigiSoft_Product_and_Services.pdf' to 'info_docs/all_data/terms and policies/DigiSoft_Product_and_Services.pdf'
Uploaded 'data/all_data/terms and policies/DigiSoft_Security_and_Compliance.

***Download S3 Folder to Local System***

In [ ]:
local_path = 'downloads'
s3_prefix = 'info_docs'

def download_directory(bucket_name, s3_prefix, local_path):
    os.makedirs(local_path, exist_ok=True)
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix + "/"):
        for obj in page.get("Contents", []):
            s3_key = obj["Key"]

            # Skip folder keys 
            if s3_key.endswith("/"):
                continue

            # Build local file path preserving folder structure under s3_prefix
            rel = os.path.relpath(s3_key, s3_prefix)
            local_file = os.path.join(local_path, rel)

            os.makedirs(os.path.dirname(local_file), exist_ok=True)

            s3.download_file(bucket_name, s3_key, local_file)

***Delete All Files in S3 Bucket***

In [ ]:
def delete_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:
        for obj in response['Contents']:
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])
            print(f"Deleted '{obj['Key']}' from bucket '{bucket_name}'")

delete_objects()

In [30]:
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'SNCWYM6G6HMPPYQG',
  'HostId': 'TgpRJj2YL3McqTD0CUsOdiw+dnIvRCYgaDGTiBzdc6ZKEY7JULcL0KPC8HQU4Bm4FHMnzavDcv8=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'TgpRJj2YL3McqTD0CUsOdiw+dnIvRCYgaDGTiBzdc6ZKEY7JULcL0KPC8HQU4Bm4FHMnzavDcv8=',
   'x-amz-request-id': 'SNCWYM6G6HMPPYQG',
   'date': 'Mon, 29 Dec 2025 20:00:22 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}